## <b><font color='darkblue'>一個 Constructor 參數很多的 class</font></b>
今天我們有一個 class <b><font color='blue'>Employee</font></b> 他有很多 attribute:

In [14]:
import dataclasses
from enum import Enum, auto

class JobType(Enum):
    CEO = auto()
    Manager = auto()
    Engineer = auto()
    HR = auto()

@dataclasses.dataclass(frozen=True)
class Employee:    
    name: str
    job: JobType
    age: int
    salary: int
    birthday: str
    address: str
    phone: str
    email: str

即使你覺得這麼長的 constructor 沒有問題 可是在創一個物件的時候 有些 attribute 是必須的 有些是不一定要有的 (比如說 `age` 跟 `birthday` 跟 `address` 不一定要提供) 那怎麼辦呢? 以 Python 語法, 這些有預設值的參數都必須放到最後面:

In [15]:
@dataclasses.dataclass(frozen=True)
class Employee:
    name: str
    job: JobType
    salary: int
    phone: str
    email: str
    age: int = -1
    address: str = 'N/A'
    birthday: str = 'N/A'

In [17]:
john = Employee(
    name = 'John',
    job = JobType.Engineer,
    salary = 123,
    phone = '0983-111-333',
    email = 'john@google.com',
    age = 41)

john

Employee(name='John', job=<JobType.Engineer: 3>, salary=123, phone='0983-111-333', email='john@google.com', age=41, address='N/A', birthday='N/A')

那現在如果我們知道 Engineer 的起薪是 30000; CEO 是 100000; HE 是 20000; Manager 是 50000, 那我們可以再減少需要輸入的參數如下:

In [47]:
@dataclasses.dataclass(frozen=True)
class Employee:
    name: str
    job: JobType
    salary: int
    phone: str
    email: str
    age: int = -1
    address: str = 'N/A'
    birthday: str = 'N/A'

    @classmethod
    def as_engineer(
        cls, name, phone, email, age, address: str = 'N/A', birthday: str = 'N/A', salary=30000):
        return Employee(
            name=name,
            job=JobType.Engineer,
            salary=salary,
            phone=phone,
            email=email,
            age=age,
            address=address,
            birthday=birthday)

    @classmethod
    def as_manager(
        cls, name, phone, email, age, address: str = 'N/A', birthday: str = 'N/A', salary=50000):
        return Employee(
            name=name,
            job=JobType.Manager,
            salary=salary,
            phone=phone,
            email=email,
            age=age,
            address=address,
            birthday=birthday)

In [48]:
john = Employee.as_engineer(
    name='John',
    phone = '0983-111-333',
    email = 'john@google.com',
    age = 41)
john

Employee(name='John', job=<JobType.Engineer: 3>, salary=30000, phone='0983-111-333', email='john@google.com', age=41, address='N/A', birthday='N/A')

In [50]:
mary = Employee.as_manager(
    name='John',
    phone = '0983-666-567',
    email = 'john@google.com',
    age = 23)
mary

Employee(name='John', job=<JobType.Manager: 2>, salary=50000, phone='0983-666-567', email='john@google.com', age=23, address='N/A', birthday='N/A')

這樣有個問題是 `address` 與 `birthday` 參數日後如果要改值成 '?', 我們就有很多地方需要去改. 而且這參數的給法不是很直覺.

In [23]:
# TypeError: Employee.as_engineer() missing 1 required positional argument: 'age'
# john = Employee.as_engineer(
#     name='John',
#     phone = '0983-111-333',
#     email = 'john@google.com')

於是我們就要進入今天的主題, 來看看怎麼使用 設計模式 [**Builder**](https://en.wikipedia.org/wiki/Builder_pattern) 套用到上面的用法.

## <b><font color='darkblue'>DP Builder</font></b>
首先來看看這個 Pattern 的定義:
> The Builder is a [**design pattern**](https://en.wikipedia.org/wiki/Software_design_pattern) designed to <b>provide a flexible solution to various object creation problems in [object-oriented  programming](https://en.wikipedia.org/wiki/Object-oriented_programming)</b>. The intent of the Builder design pattern is to <b>[separate](https://en.wikipedia.org/wiki/Separation_of_concerns) the construction of a complex object from its representation</b>. It is one of the Gang of Four [**design pattern**](https://en.wikipedia.org/wiki/Software_design_pattern).<br/><br/>
> -- from [Wiki](https://en.wikipedia.org/wiki/Builder_pattern)

接著是它的 class dia

![class diagram](images/builder_class_diagram.PNG)gram:

接著我們回到類別 <b><font color='blue'>Employee</font></b> 上, 透過 Builder 設計模式, 我們可以改成:

In [43]:
@dataclasses.dataclass(frozen=True)
class Employee:
    name: str
    job: JobType
    salary: int
    phone: str
    email: str
    age: int
    address: str
    birthday: str
    
    class _Builder:
        name: str
        job: JobType
        salary: int
        phone: str
        email: str
        age: int = -1
        address: str = 'N/A'
        birthday: str = 'N/A'

        def build(self) -> Employee:
            return Employee(
                name = self.name,
                job = self.job,
                salary = self.salary,
                phone = self.phone,
                email = self.email,
                age = self.age,
                address = self.address,
                birthday = self.birthday)
        
        def set_name(self, name: str):
            self.name = name
            return self

        def set_job(self, job: JobType):
            self.job = job
            return self

        def set_salary(self, salary: int):
            self.salary = salary
            return self

        def set_phone(self, phone: str):
            self.phone = phone
            return self

        def set_email(self, email: str):
            self.email = email
            return self

        def set_age(self, age: int):
            self.age = age
            return self

        def set_address(self, address: str):
            self.address = address
            return self

        def set_birthday(self, birthday: str):
            self.birthday = birthday
            return self

    @classmethod
    def as_engineer(cls) -> Employee._Builder:
        builder = Employee._Builder()
        builder.set_job(JobType.Engineer)
        builder.set_salary(30000)
        return builder

    @classmethod
    def as_manager(cls) -> Employee._Builder:
        builder = Employee._Builder()
        builder.set_job(JobType.Manager)
        builder.set_salary(50000)
        return builder

In [41]:
john = (
    Employee.as_engineer()
        .set_name('John')
        .set_phone('0983-111-333')
        .set_email('john@google.com')).build()

john

Employee(name='John', job=<JobType.Engineer: 3>, salary=30000, phone='0983-111-333', email='john@google.com', age=-1, address='N/A', birthday='N/A')

In [45]:
mary = (
    Employee.as_manager()
        .set_name('Mary')
        .set_phone('0983-111-333')
        .set_email('mary@google.com')
        .set_age(23)).build()

mary

Employee(name='Mary', job=<JobType.Manager: 2>, salary=50000, phone='0983-111-333', email='mary@google.com', age=23, address='N/A', birthday='N/A')

## <b><font color='darkblue'>Supplement</font></b>
* [ithelp - 隨著自己想要的客製化 - 建造者模式 (Builder Pattern)](https://ithelp.ithome.com.tw/articles/10204732)
> 建築者模式是設計來提供一個有彈性解決方案，用OOP的方式來解決一個不同(複雜)物件的創造。目的是為了要分離一個複雜物品的建造和表示建造的方式。
* [medium - 設計模式—建造者模式 (Builder Design Pattern)](https://medium.com/wenchin-rolls-around/%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F-%E5%BB%BA%E9%80%A0%E8%80%85%E6%A8%A1%E5%BC%8F-builder-design-pattern-7c8eac7c9a7)
> 建造者 (Builder) 模式將物件的「建構」與「表示」分離，隱藏並封裝建構過程的細節。它讓我們可以將物件本身拆解成不同的元件，一步一步建造每一部分，最後產生出我們想要的複雜物件。
* [Refactor Guru - Builder](https://refactoring.guru/design-patterns/builder)
> <b>Builder</b> is a creational design pattern that lets you construct complex objects step by step. The pattern allows you to produce different types and representations of an object using the same construction code.